<a href="https://colab.research.google.com/github/BankNatchapol/Comparison-Of-Quantum-Gradient/blob/main/concept_implementation/pennylane_qng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane

     |████████████████████████████████| 580 kB 5.0 MB/s 


In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import GradientDescentOptimizer, QNGOptimizer

import pandas as pd

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
num_wires = 1
dev = qml.device("default.qubit", wires=num_wires)

In [ ]:
def parameter_shift_term(qnode, params, i, j):
    
    shifted = params.copy()
    
    shifted[i, j] += np.pi/2
    forward = qnode(shifted)  # forward evaluation

    shifted[i, j] -= np.pi
    backward = qnode(shifted) # backward evaluation

    return 0.5 * (forward - backward)

def parameter_shift(qnode, params):
    gradients = np.zeros_like((params))
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):
            gradients[i, j] += parameter_shift_term(qnode, params, i, j)

    return gradients

In [ ]:
# problem gate 
def problem():
    qml.U3(1.44, 0.8, 2.1, wires=0)

In [ ]:
# guesting ansatz state
def ansatz(var):
    for wire in range(num_wires):
      qml.Hadamard(wires=wire)
      qml.RX(var[0+wire], wires=wire)
      qml.RY(var[1+wire], wires=wire)
      qml.RZ(var[2+wire], wires=wire)

In [ ]:
# objective function
@qml.qnode(dev)
def cost_function(var):
    for v in var: 
      ansatz(v)

    problem() # problem gate 

    return qml.expval(qml.Projector([1],wires=0)) # get amplitude of of |1>

In [ ]:
# target result of problem gate
@qml.qnode(dev)
def target():
    problem()
    return qml.probs(wires=[0])  # get target probability

In [ ]:
# prediction circuit
@qml.qnode(dev)
def prediction(var):
    for v in reversed(var):
      qml.adjoint(ansatz)(v)
    return qml.probs(wires=[0])  # get prediction probability

In [ ]:
print("Target state: ", target())

Target state:  [0.56521185 0.43478815]


In [ ]:
np.random.seed(1)
num_layers = 2
var_init = 0.05*np.random.randn(num_layers, 3*num_wires)

In [ ]:
print("Initial cost: ", cost_function(var_init))

Initial cost:  0.38420647954293236


# **Quantum Natural Gradient(QNG)**
Quantum Fisher Information(QFI) is the quantum analogue of classical Fisher Information that is a way of measuring the amount of information that an observable random variable X carries about an unknown parameter θ upon which the probability of X depends.<br>
By using Pennylane library, we can create QFI using Block diagonal and Diagonal Approximation method with qml.metric_tensor function.<br><br>
$$QFI = qml.metric\_tensor(quantum\_circuit)$$<br>
we can calculate Quantum Natural Gradient using inverse Quantum Fisher Information metric<br><br>
$$QNG = QFI^{-1} \nabla J(\theta)$$<br>
Calculate inverse Fisher Information metric $QFI^{-1}$ using numpy linear solver<br><br>
$$QFIx = \nabla J(\theta) \quad:\quad find \quad x$$ 

In [ ]:
@qml.qnode(dev)
def metric_tensor_circuit(var, wires=0):
    for v in var: 
      ansatz(v)
    problem() # problem gate 
    return qml.expval(qml.PauliX(wires=0))

metric_fn = qml.metric_tensor(metric_tensor_circuit)
print(metric_fn(var_init))

[[1.11022302e-16 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.50000000e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.49766169e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 2.49766169e-01
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.48402634e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 2.95354196e-03]]


In [ ]:
opt = qml.QNGOptimizer(0.01)

var = var_init.copy()
loss_plot_QNG = []

for it in range(501):# while True:
    var, _cost = opt.step_and_cost(lambda v: cost_function(v), var, 
                                   grad_fn=lambda var: parameter_shift(cost_function, var),
                                   metric_tensor_fn=qnodes.qnodes[0].metric_tensor) 
    loss_plot_QNG.append(_cost)

    if it%100==0:
      print("Iter: {:5d} | Cost: {:0.15f} ".format(it, _cost))

Iter:     0 | Cost: 0.384206479542932 
Iter:   100 | Cost: 0.000044607430908 
Iter:   200 | Cost: 0.000000003537183 
Iter:   300 | Cost: 0.000000000000398 
Iter:   400 | Cost: 0.000000000000000 
Iter:   500 | Cost: 0.000000000000000 


In [ ]:
opt = qml.GradientDescentOptimizer(0.01)

def grad_fn(var):
  grad = parameter_shift(cost_function, var)
  grad_flatten = grad.flatten()
  metric_tensor = metric_fn(var)
  return np.linalg.solve(metric_tensor, grad_flatten)

var_gds = var_init.copy()
loss_plot_GDS = []

for it in range(501):# while True:
    var_gds, _cost = opt.step_and_cost(lambda v: cost_function(v), var_gds, 
                                   grad_fn=grad_fn)
    loss_plot_GDS.append(_cost)

    if it%100==0:
      print("Iter: {:5d} | Cost: {:0.15f} ".format(it, _cost))

Iter:     0 | Cost: 0.384206479542932 
Iter:   100 | Cost: 0.000044607430908 
Iter:   200 | Cost: 0.000000003537183 
Iter:   300 | Cost: 0.000000000000398 
Iter:   400 | Cost: 0.000000000000000 
Iter:   500 | Cost: 0.000000000000000 


In [ ]:
#@title 
qng = pd.DataFrame({"Iteration":range(len(loss_plot_QNG)), "Loss":loss_plot_QNG})
gds = pd.DataFrame({"Iteration":range(len(loss_plot_GDS)), "Loss":loss_plot_GDS})

fig = go.Figure()
fig.add_trace(go.Scatter(x=qng["Iteration"], y=qng["Loss"], mode="lines", name="Pennyalne's implementation"))
fig.add_trace(go.Scatter(x=gds["Iteration"], y=gds["Loss"], mode="lines", name="Concept's implementation"))
fig.update_layout(title_text="Pennyalne's vs Concept's implementation")
fig.show()

In [ ]:
print("Target probs    : ", target())
print("Prediction probs: ", prediction(var_gds))

Target probs    :  [0.56521185 0.43478815]
Prediction probs:  [0.56521185 0.43478815]
